# Lab 9.7 - About Dropout with MNIST Classifier

**Jonathan Choi 2021**

**[Deep Learning By Torch] End to End study scripts of Deep Learning by implementing code practice with Pytorch.**

If you have an any issue, please PR below.

[[Deep Learning By Torch] - Github @JonyChoi](https://github.com/jonychoi/Deep-Learning-By-Torch)

Here, we are going to adapt the new module, **Dropout** that makes the neural network more compatible to the general data, one of the techniques to prevent overfitting as a regularization. We are going to create the multi layer neural network as always, additional with Dropout.

## Imports

In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random

In [57]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(1)
torch.manual_seed(1)

if device == 'cuda':
    torch.cuda.manual_seed_all(1)

In [58]:
mnist_train = datasets.MNIST(root = 'MNIST_data/',
                             download = True,
                             transform = transforms.ToTensor(),
                             train=True)
mnist_test = datasets.MNIST(root = 'MNIST_data/',
                            download = True,
                            transform = transforms.ToTensor(),
                            train = False)

In [59]:
training_epochs = 15
batch_size = 100
learning_rate = 0.001
drop_prob = 0.3

In [60]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, shuffle = True, drop_last = True, batch_size = batch_size)

In [61]:
class Dropout_MNIST_Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.sq = nn.Sequential(
            nn.Linear(784, 516),
            nn.ReLU(),
            nn.Dropout(p=drop_prob),
            nn.Linear(516, 516),
            nn.ReLU(),
            nn.Dropout(p=drop_prob),
            nn.Linear(516, 516),
            nn.ReLU(),
            nn.Dropout(p=drop_prob),
            nn.Linear(516, 516),
            nn.ReLU(),
            nn.Dropout(p=drop_prob),
            nn.Linear(516, 10)
        )
        self.weightInitializer()

    def weightInitializer(self):
        for index, layer in enumerate(self.sq):
            if index != 1 and index != 2 and index != 4 and index != 5 and index != 7 and index != 8 and index != 10 and index != 11:
                nn.init.xavier_uniform_(layer.weight)

    def forward(self, x):
        return self.sq(x)

In [62]:
model = Dropout_MNIST_Classifier().to(device)

In [63]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [64]:
total_batch = len(data_loader)

for epoch in range(training_epochs):

    avg_cost = 0

    for X, Y in data_loader:

        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        #prediction
        pred = model(X)

        #cost
        cost = F.cross_entropy(pred, Y)

        #Reduce the cost
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost

    avg_cost = avg_cost / total_batch

    print('Epoch: {:d} / 15, Cost: {:.6f}'.format(epoch + 1, avg_cost.item()))

Epoch: 1 / 15, Cost: 0.309588
Epoch: 2 / 15, Cost: 0.142839
Epoch: 3 / 15, Cost: 0.112396
Epoch: 4 / 15, Cost: 0.092985
Epoch: 5 / 15, Cost: 0.082222
Epoch: 6 / 15, Cost: 0.075470
Epoch: 7 / 15, Cost: 0.065344
Epoch: 8 / 15, Cost: 0.065210
Epoch: 9 / 15, Cost: 0.057394
Epoch: 10 / 15, Cost: 0.051965
Epoch: 11 / 15, Cost: 0.053811
Epoch: 12 / 15, Cost: 0.049630
Epoch: 13 / 15, Cost: 0.047635
Epoch: 14 / 15, Cost: 0.043034
Epoch: 15 / 15, Cost: 0.045472


### Take a moment!

```model.eval()``` should be exist at the first line of inference code to disable the dropout => dropout = False

In [72]:
#Prediction and accuracy check

with torch.no_grad():
    model.eval()

    X = mnist_test.data.view(-1, 28 * 28).float().to(device)
    Y = mnist_test.targets.to(device)

    #prediction
    prediction = torch.argmax(model(X), 1)
    correct_prediction = (prediction == Y)
    accuracy = correct_prediction.float().mean()

    #check samples and prediction
    r = random.randint(0, len(mnist_test) - 1)
    X_single_prediction = X[r]
    Y_single_prediction = Y[r]

    print('Accuracy: {:.9f}'.format(accuracy))
    print('Label: {}, Prediction: {}'.format(Y_single_prediction, torch.argmax(model(X_single_prediction))))

Accuracy: 0.981099963
Label: 0, Prediction: 0
